1. preprocess:按照rank的方式进行排序（参考geneformer）
2. tokenizer构建，直接基于前面的预料库构建即可
3. BERT架构构建（或者其他的）,MLM
4. 预训练

## preprocessing

In [5]:
# TODO: eid信息缺失了，并且部分蛋白是用mean填充的，后续可以根本不需要填充，因为可以传入的时候没有他们
import pandas as pd

train_data = pd.read_pickle("result/part1/train_imputed.pkl").set_index("eid")
test_data = pd.read_pickle("result/part1/test_imputed.pkl").set_index("eid")


protein_cols = test_data.columns[test_data.columns.tolist().index("C3") :].tolist()
train_data = train_data[protein_cols]
test_data = test_data[protein_cols]

In [10]:
total = pd.concat([train_data, test_data], axis=0)
train_data = total.iloc[4000:]
eval_data = total.iloc[:4000]
eval_data

,C3,KLK7,GCHFR,NHLRC3,APOD,GAPDH,TP53I3,CPA4,ANXA2,GRSF1,...,EGFR,TGFBR3,CRTAC1,IGFBP7,SELE,VWF,NOTCH3,CNTN1,ENG,ICAM2
eid,,,,,,,,,,,,,,,,,,,,,
1982486.0,0.040385,0.007345,0.057373,-0.000238,0.02491,-0.065304,0.02669,-0.009212,0.073499,0.082175,...,0.571850,0.5713,-0.663750,0.777200,0.619300,0.812200,0.791950,0.148600,0.127500,1.582800
5937359.0,0.040385,0.007345,0.057373,-0.000238,0.02491,-0.065304,0.02669,-0.009212,0.073499,0.082175,...,-0.064050,-0.3107,-0.013850,-0.773100,0.335000,0.691400,0.309750,0.085900,-0.116300,0.360600
3737374.0,-0.557200,-0.499600,0.575800,-0.589900,-0.49640,0.118850,0.17070,-0.074400,0.032050,-0.209000,...,-0.078600,-0.0684,0.334750,0.429200,-0.475600,0.707900,-0.119200,-0.336500,0.256400,0.169300
5525566.0,0.141400,-0.525100,-0.338000,-0.233100,-0.02070,0.077700,0.33230,0.144400,2.277800,1.125300,...,-0.001350,-0.4087,0.010150,-0.213000,0.046050,0.902600,0.043200,0.076700,0.116900,0.158500
1828132.0,0.386100,-0.094400,0.546200,-0.163200,0.02491,0.088000,0.02180,0.194000,-0.261200,-0.168600,...,-0.004284,-0.0087,-0.029539,0.022568,-0.027118,0.008048,0.004249,0.000619,0.001707,-0.026825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200051.0,-0.190400,-0.538750,0.113200,0.164100,-0.58220,-0.072700,-0.72350,-0.216350,-0.083400,-0.599400,...,-0.225700,0.3121,0.121850,0.236300,0.301800,-0.237100,-0.033300,-0.146900,0.054100,0.277500
4095573.0,-0.086900,-0.232300,-0.814900,-0.269100,0.00000,-0.551450,-0.70850,-0.110800,0.381650,-0.291400,...,0.129700,-0.3872,-1.201150,-0.440900,-1.261200,-0.007300,-0.057200,-0.028600,-0.251700,-0.519800
5600706.0,0.285500,-0.227300,0.352500,0.293800,-0.27210,0.109750,0.63860,-0.117400,0.588050,0.786700,...,0.003500,-0.0264,-0.566850,-0.061300,0.409400,-0.431800,0.300300,0.085100,-0.099600,0.358300


In [12]:
train_data.to_pickle("result/dl/ProteomicsBERT/train.pkl")
eval_data.to_pickle("result/dl/ProteomicsBERT/eval.pkl")

In [16]:
from datasets import Dataset
from collections import defaultdict


def df2dataset(df, tokenizer, max_length=2048):
    res = defaultdict(list)
    for idx, row in df.iterrows():
        ranked_row = row.sort_values(ascending=False).dropna()

        res["eid"].append(ranked_row.name)
        res["proteins"].append(" ".join(ranked_row.index.tolist()))
        res["values"].append(ranked_row.values.tolist())
    dataset = Dataset.from_dict(res)

    def group_texts(examples):

        tokenized_inputs = tokenizer(
            examples["proteins"],
            return_special_tokens_mask=True,
            add_special_tokens=True,
            padding="max_length",
            max_length=max_length,
            truncation=True,
            truncation_strategy="only_last",
        )

        return tokenized_inputs

    dataset = dataset.map(
        group_texts,
        batched=True,
        remove_columns=["proteins"],
        num_proc=8,
    )
    return dataset

In [17]:
from transformers import (
    BertTokenizer,
    BertTokenizerFast,
    PreTrainedTokenizerFast,
    AutoTokenizer,
)


tokenizer = AutoTokenizer.from_pretrained("transtab/tokenizer")

In [18]:
max_length = tokenizer.model_max_length if tokenizer.model_max_length <= 1e5 else 2048

train_dataset = df2dataset(train_data, max_length=max_length, tokenizer=tokenizer)
print(
    f"train_dataset: {train_dataset} with input_ids fix length: {len(train_dataset[0]['input_ids'])}"
)

test_dataset = df2dataset(test_data, max_length=max_length, tokenizer=tokenizer)
print(
    f"test_dataset: {test_dataset} with input_ids fix length: {len(test_dataset[0]['input_ids'])}"
)

Map (num_proc=8):   0%|          | 0/47439 [00:00<?, ? examples/s]

TimeoutError: 

In [2]:
from datasets import Dataset
from collections import defaultdict

res = defaultdict(list)

for idx, row in test_data.iterrows():
    ranked_row = row.sort_values(ascending=False).dropna()

    res["eid"].append(ranked_row.name)
    res["proteins"].append(" ".join(ranked_row.index.tolist()))
    res["values"].append(ranked_row.values.tolist())

test_dataset = Dataset.from_dict(res)
test_dataset

Dataset({
    features: ['eid', 'proteins', 'values'],
    num_rows: 4000
})

In [3]:
from transformers import (
    BertTokenizer,
    BertTokenizerFast,
    PreTrainedTokenizerFast,
    AutoTokenizer,
)


tokenizer = AutoTokenizer.from_pretrained("transtab/tokenizer")

In [4]:
from transformers import AutoTokenizer
import multiprocessing
from transformers import BertTokenizer, BertTokenizerFast, PreTrainedTokenizerFast


def group_texts(examples, max_length=2048):

    tokenized_inputs = tokenizer(
        examples["proteins"],
        return_special_tokens_mask=True,
        add_special_tokens=True,
        padding="max_length",
        max_length = max_length
        truncation=True,
        truncation_strategy="only_last",
    )

    return tokenized_inputs


# preprocess dataset
tokenized_datasets = test_dataset.map(
    group_texts,
    batched=True,
    remove_columns=["proteins"],
    num_proc=8,
)

Map (num_proc=8):   0%|          | 0/4000 [00:00<?, ? examples/s]

In [5]:
tokenized_datasets

Dataset({
    features: ['eid', 'values', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
    num_rows: 4000
})

In [6]:
from transformers import DataCollatorForLanguageModeling


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [1]:
from transformers import BertConfig

config = BertConfig()

config

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.39.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [2]:
tokenizer.vocab_size

NameError: name 'tokenizer' is not defined

In [3]:
bertconfig = BertConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=256,
    num_attention_heads=8,
    intermediate_size=512,
    max_position_embeddings=tokenizer.model_max_length,
    num_hidden_layers=6,
)

NameError: name 'tokenizer' is not defined

In [10]:
from transformers import AutoModelForMaskedLM, BertForMaskedLM

model = BertForMaskedLM(bertconfig)

In [11]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(2919, 256, padding_idx=0)
      (position_embeddings): Embedding(2911, 256)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=256, out_features=256, bias=True)
              (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_aff

In [12]:
from transformers import Trainer, TrainingArguments

In [13]:
training_args = TrainingArguments(
    output_dir="test",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
    data_collator=data_collator,
)

# trainer.train()

/home/xutingfeng/miniforge3/envs/rapids-24.02/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [14]:
trainer.train_dataset

Dataset({
    features: ['eid', 'values', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
    num_rows: 4000
})

In [15]:
for batch in trainer.get_train_dataloader():

    break

In [16]:
batch.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [17]:
batch["token_type_ids"]

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0')

In [18]:
batch["input_ids"]

tensor([[2540, 1384,  803,  ...,    0,    0,    0],
        [1957, 2635, 1952,  ...,    0,    0,    0],
        [2778, 2540, 1377,  ...,    0,    0,    0],
        ...,
        [   3, 1146, 1248,  ..., 2204,  910, 2084],
        [1234,    3, 2551,  ...,   15, 2738, 2444],
        [   3, 2144, 1354,  ..., 1028, 1571, 1230]], device='cuda:0')

In [19]:
batch["input_ids"][0][:20]

tensor([2540, 1384,  803, 2042,   66, 1375,  380,    3,  364, 1886, 1232, 1215,
        2176, 1412, 2706,  963, 2251, 1254, 2118, 2911], device='cuda:0')

In [20]:
batch["labels"][0][:20]

tensor([-100, -100, -100, -100, -100, -100, -100, 2336, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100], device='cuda:0')

In [21]:
batch["input_ids"].shape

torch.Size([8, 2911])

In [22]:
o = model(**batch)

In [27]:
o.logits.shape

torch.Size([8, 2911, 2919])

In [ ]:
from transformers import AutoConfig


config = AutoConfig.from_pretrained(
    "distilbert/distilroberta-base",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

In [ ]:
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("distilbert/distilroberta-base")

In [ ]:
data_collator

In [ ]:
from transformers import BertTokenizer, BertTokenizerFast, PreTrainedTokenizerFast


protein_tokenizer = BertTokenizerFast.from_pretrained("transtab/tokenizer")


def tokenize_function(examples):
    return protein_tokenizer(examples["proteins"])


tokenized_datasets = test_dataset.map(
    tokenize_function, batched=True, num_proc=4, batch_size=32
)

In [ ]:
protein_tokenizer(test_dataset[0]["proteins"])

In [ ]:
# res_list = {}


# for idx, row in test_data.set_index("eid").iterrows():
#     ranked_row = row.sort_values(ascending=False).dropna()
#     res = {}
#     res["eid"] = ranked_row.name
#     res["proteins"] = ranked_row.index.tolist()
#     res["values"] = ranked_row.values.tolist()
#     res_list.append(res)


# def dict_generator():
#     for i in range(len(res_list)):
#         yield res_list[i]

# Dataset.from_generator(dict_generator, num_proc=4)

In [ ]:
Dataset.from_generator(res_list)

In [ ]:
import numpy as np


def rank_genes(gene_vector, gene_tokens):
    """
    Rank gene expression vector.
    """
    # sort by median-scaled gene values
    sorted_indices = np.argsort(-gene_vector)
    return gene_tokens[sorted_indices], gene_vector[sorted_indices]


def rank_sorted(examples, protein_cols=None):
    """
    Rank protein expression vectors.
    """

    protein_expression_vectors = np.array([examples[col] for col in protein_cols])
    protein_tokens = protein_cols
    ranked_protein_tokens, ranked_protein_expression_vectors = rank_genes(
        protein_expression_vectors, protein_tokens
    )

    other_cols = [col for col in examples.columns if col not in protein_cols]
    return_res = {}
    for i, col in enumerate(other_cols):
        return_res[col] = examples[col].values
    return_res["protein_tokens"] = ranked_protein_tokens
    return_res["protein_expression_vectors"] = ranked_protein_expression_vectors
    return return_res


test_dataset.map(lambda x: rank_sorted(x, protein_cols=protein_cols))

In [ ]:
# normalize = True 
# if normalize:
#     # pd.concat([train_data, test_data])[protein_cols]